# 엑셀 데이터프레임 실습 노트북

이 노트북에서는 다운로드한 poshThru_ 엑셀 파일을 데이터프레임으로 변환하고, 다양한 데이터 전처리 및 분석 실습을 진행합니다.

In [1]:
# 1. 필요한 라이브러리 임포트
import os
import glob
import pandas as pd

In [33]:
# 2. 엑셀 파일에서 데이터프레임으로 변환
# 다운로드 폴더에서 poshThru_로 시작하는 최신 엑셀 파일 찾기
download_dir = r"C:\\Users\\pcuser\\Downloads"
pattern = os.path.join(download_dir, "poshThru_*.xls*")
files = glob.glob(pattern)
if not files:
    raise FileNotFoundError("다운로드된 poshThru_ 파일을 찾을 수 없습니다.")
latest_file = max(files, key=os.path.getctime)
print(f"최신 다운로드 파일: {latest_file}")
# PO1 No, SO No.를 문자열로, Ship 컬럼은 Int64로 강제 변환하여 읽기
col_converters = {
    'PO1 No': lambda x: '' if pd.isnull(x) else str(int(float(x))) if str(x).replace('.','',1).isdigit() and float(x).is_integer() else str(x),
    'SO No.': lambda x: '' if pd.isnull(x) else str(int(float(x))) if str(x).replace('.','',1).isdigit() and float(x).is_integer() else str(x),
    'Ship': lambda x: None if pd.isnull(x) else int(float(x)) if str(x).replace('.','',1).isdigit() and float(x).is_integer() else x
}
df = pd.read_excel(latest_file, converters=col_converters)
df['Ship'] = df['Ship'].astype('Int64')
df.head()

최신 다운로드 파일: C:\\Users\\pcuser\\Downloads\poshThru_20251217101620.xls


,Biz Type,Week,Model,PO No.,Ship To,Rev. ORG,PO Status,Shipping,Receipt,Cancel,...,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61
0,Thru,2025-W47,24BA450-B.AEKQ,JGMZ25027297,EEUK,E5I,NaN,Y,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Thru,2025-W36,24BA450-B.AEUQ,JGMZ25020172,EEPT,E8J,NaN,Y,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Thru,2025-W40,24BA450-B.AEUQ,JGMZ25023051,EEHS,E6K,NaN,Y,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Thru,2025-W40,24BA450-B.AEUQ,JGMZ25023113,EEES,E7M,NaN,Y,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Thru,2025-W42,24BA450-B.AEUQ,JGMZ25024468,EEIS,E7J,NaN,Y,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = df[['Model', 'PO No.', 'Ship To', 'Shipping', 'Cancel', 'PO', 'OQC Report', 'OQC Date', 'OQC Result', 'Ship', 'Issued Date', 'RSD','Ship Date', 'BL No', 'BL Status', 'Method', 'Price Term', 'Unit Price', 'Currency', 'Payment Term', 'PO1 No', 'SO No.', 'Final Destination']]
df = df[~df['PO No.'].isin(['JGMZ23029072', 'JGMZ23027440', 'JGMZ23025674', 'JGMZ23029071', 'JGMZ23025215', 'JGMZ23024615', 'JGMZ24029466', 'JGMZ24029450', 'JGMZ24026061',
                                'JGMZ24026835', 'JGMZ24030939', 'JGMZ24038096', 'JGMZ24038150', 'JGMZ24038785', 'JGMZ24038783', 'JGMZ24038784', 'JGMZ25024444', 'JGMZ25024445'])] # RMA후 기존 PO 삭제가 안되어 강제로 제거 필요

In [35]:
df['Ship'].unique()

<IntegerArray>
[ 336, 1096,  288,  432,  144, <NA>,   96,  192,  300, 1176, 2352, 3960, 4704,
  240,  748,   48,  840, 3528, 2184,  744, 1008,  168, 1064,  104,  420,  208,
  572, 1320,  396,  108,  180,   72, 2016, 3024,  360,   84,  486,   42,  324,
  798, 2772,  924, 2160, 1080, 3240, 4320,  195,  234,  585,  273,  429, 1287,
 2589,    1]
Length: 54, dtype: Int64

In [ ]:
# 4. 결측치 처리 예시
# 결측치 개수 확인
df.isnull().sum()
# 결측치가 있는 행 제거 예시
df_dropna = df.dropna()
# 결측치를 0으로 대체 예시
df_fillna = df.fillna(0)
df_fillna.head()

In [ ]:
# 5. 데이터 타입 변환 예시
# 예: 'Qty' 컬럼을 정수형으로 변환
df['Qty'] = df['Qty'].astype('Int64')
df.dtypes

In [ ]:
# 6. 특정 컬럼 값 변환 및 정제 예시
# 예: 'Status' 컬럼의 'Complete'를 1, 'Incomplete'를 0으로 변환
df['Status_num'] = df['Status'].replace({'Complete': 1, 'Incomplete': 0})
df[['Status', 'Status_num']].head()

In [ ]:
# 7. 새로운 컬럼 생성 및 파생 변수 추가 예시
# 예: 'Qty'와 'PO' 컬럼의 차이를 나타내는 'Qty_PO_Diff' 파생 변수 생성
df['Qty_PO_Diff'] = df['Qty'] - df['PO']
df[['Qty', 'PO', 'Qty_PO_Diff']].head()